In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Hotel reservations") \
  .getOrCreate()

25/09/01 08:46:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
data = spark.read.csv("data/hotel_reservations.csv", header=True, inferSchema=True)

data.printSchema()

root
 |-- Booking_ID: string (nullable = true)
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- type_of_meal_plan: string (nullable = true)
 |-- required_car_parking_space: integer (nullable = true)
 |-- room_type_reserved: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_year: integer (nullable = true)
 |-- arrival_month: integer (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- market_segment_type: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- no_of_previous_cancellations: integer (nullable = true)
 |-- no_of_previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: integer (nullable = true)
 |-- booking_status: string (nullable = true)



In [4]:
data.count()

36275

In [5]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())                                          

Train size:  29040
Test size:  7235


In [6]:
train_data.show()

+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+------------------+----------------------+--------------+
|Booking_ID|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|type_of_meal_plan|required_car_parking_space|room_type_reserved|lead_time|arrival_year|arrival_month|arrival_date|market_segment_type|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|booking_status|
+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+--

In [7]:
pipeline_stages = []

In [8]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'arrival_month', 'market_segment_type']

for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol= col + '_index')
    print(f'StringIndexer {string_indexer.getInputCol()} -> {string_indexer.getOutputCol()}')

    encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(), outputCol=col + '_vec', dropLast=False)
    print(f'OneHotEncoder {encoder.getInputCol()} -> {encoder.getOutputCol()}')
    print()

    pipeline_stages += [string_indexer, encoder]

StringIndexer type_of_meal_plan -> type_of_meal_plan_index
OneHotEncoder type_of_meal_plan_index -> type_of_meal_plan_vec

StringIndexer room_type_reserved -> room_type_reserved_index
OneHotEncoder room_type_reserved_index -> room_type_reserved_vec

StringIndexer arrival_month -> arrival_month_index
OneHotEncoder arrival_month_index -> arrival_month_vec

StringIndexer market_segment_type -> market_segment_type_index
OneHotEncoder market_segment_type_index -> market_segment_type_vec



In [9]:
pipeline_stages += [StringIndexer(inputCol='booking_status', outputCol='booking_status_index')]

In [10]:
encoded_categorical_cols = [col + "_vec" for col in categorical_cols]
encoded_categorical_cols

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec']

In [11]:
numeric_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
               'required_car_parking_space', 'lead_time',
               'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
               'avg_price_per_room', 'no_of_special_requests'] 

In [12]:
input_columns = encoded_categorical_cols + numeric_cols
input_columns

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'required_car_parking_space',
 'lead_time',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [13]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_columns, outputCol='features')

pipeline_stages.append(assembler)

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(featuresCol='features', labelCol='booking_status_index')

pipeline_stages.append(dtc)

In [15]:
pipeline_stages

[StringIndexer_e1ce7a965494,
 OneHotEncoder_a176f56222b5,
 StringIndexer_ff7aa6547196,
 OneHotEncoder_42af1b483615,
 StringIndexer_05cf5c24719c,
 OneHotEncoder_78a9cda439ce,
 StringIndexer_1f1eb4381aa9,
 OneHotEncoder_ef98fcf9fc2b,
 StringIndexer_5377333499e8,
 VectorAssembler_a42f14d985f0,
 DecisionTreeClassifier_2fe9ae61842a]

In [16]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=pipeline_stages)

In [17]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
  .addGrid(dtc.maxDepth, [2, 5, 10]) \
  .addGrid(dtc.maxBins, [10, 20, 40]) \
  .addGrid(dtc.minInstancesPerNode, [1, 2, 4]) \
  .build()


evaluator = MulticlassClassificationEvaluator(
    labelCol='booking_status_index',
    predictionCol='prediction',
    metricName='accuracy'
)

tvs_model = TrainValidationSplit(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    seed=42
)

best_model = tvs_model.fit(train_data)

25/09/01 08:47:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
predictions = best_model.transform(test_data)

In [19]:
accuracy = evaluator.evaluate(predictions) * 100
print(f'Accuracy = {accuracy:.2f}%')

Accuracy = 86.19%


In [20]:
for param_values, metric_value in zip(paramGrid, best_model.validationMetrics):
    for param, value in param_values.items():
        print(f'{param.name} = {value}')
    print(f'Accuracy -> {metric_value}')

maxDepth = 2
maxBins = 10
minInstancesPerNode = 1
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 10
minInstancesPerNode = 2
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 10
minInstancesPerNode = 4
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 20
minInstancesPerNode = 1
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 20
minInstancesPerNode = 2
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 20
minInstancesPerNode = 4
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 40
minInstancesPerNode = 1
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 40
minInstancesPerNode = 2
Accuracy -> 0.7668415240198785
maxDepth = 2
maxBins = 40
minInstancesPerNode = 4
Accuracy -> 0.7668415240198785
maxDepth = 5
maxBins = 10
minInstancesPerNode = 1
Accuracy -> 0.8256488128106019
maxDepth = 5
maxBins = 10
minInstancesPerNode = 2
Accuracy -> 0.8256488128106019
maxDepth = 5
maxBins = 10
minInstancesPerNode = 4
Accuracy -> 0.8256488128106019
maxDepth = 5
maxBins = 20
mi